# Exemplo 07: Processamento e Leitura Paralela no Spark
## Contagem de Palavras Serial & Paralelo

A contagem de palavras em um texto é um dos exemplos mais comuns paa avaliar o desempenho de aplicações distribuídas. Ao contrário do exemplo de cálculo do número Pi que exige apenas processamento, a contagem de palavras exige a leitura de um arquivo texto. Assim, é possível avaliar o desempenho de aplicações paralelas com leitura de arquivo.

A contagem pode ser realizada através do paradgma MapReduce. A função Map em um pedaço do texto separa todas as palavras e contabiliza a ocorrencia de cada uma. A função Reduce irá juntar a contagem de todas as partes e totalizar o número de ocorrência de cada palavra.

Neste exemplo vamos contar as palavras do livro _Guerra e Paz_ de Tolstói (em ingles War and Peace, do projeto Gutemberg).

No primeiro caso, contamos em um único host, o segundo caso faz a contagem paralela em vários hosts usando virtualização JVM mas com uma partição única do arquivo, e o terceiro com o arquivo de entrada dividido em partes. O quarto caso faz a contagem  paralela em vários hosts usando virtualização Docker e o quinto também Docker com o arquivo de entrada dividido em partes.

Definir o número de partes no qual o arquivo será dividido é um desafio, mas os melhores resultados são obtidos quando o número de partes é maiso ou menos igual ao número de hosts. 

Lembramos que para executar um processamento distribuído com leitura de arquivo, este deverá estar disponível em todos os hosts. A melhor forma é colocaá-lo em um sistema de arquivos distribuído.

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import time

In [2]:
# Configuration
# Processamento paralelo que le um arquivo necessita que ele esteja em todos os nós ou sistema de arquivos distribuidos
book_folder = "/data/dataset/books/english/"
book_file = "file://"+book_folder+"War_and_Peace.txt"
print(book_file)

file:///data/dataset/books/english/War_and_Peace.txt


## Word Count Serial

In [3]:
start_time = time.time()

# Create local Spark session
spark = SparkSession.builder \
        .appName("WordCountSerial") \
        .master("local[2]") \
        .getOrCreate()

# create the Spark Context
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/17 14:43:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Read de book file 
text_file = sc.textFile(book_file).cache()

counts = text_file.flatMap(lambda line: line.split()) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

counts.toDF(["Word", "Count"]).sort("Count",ascending=False).show(5)

print("--- Execution time: %s seconds ---" % (time.time() - start_time))

+----+-----+
|Word|Count|
+----+-----+
| the|31704|
| and|20564|
|  to|16320|
|  of|14855|
|   a|10018|
+----+-----+
only showing top 5 rows

--- Execution time: 9.188942193984985 seconds ---


In [5]:
spark.stop()

## Word Count Parallel Standalone Cluster

### One Data Partition

In [6]:
# Starting Spark Cluster

spark = SparkSession.builder \
        .master("spark://lasidcluster:7077") \
        .appName("WordCountParallel") \
        .config("spark.sql.shuffle.partitions","10000") \
        .config("spark.driver.memory","4g") \
        .config("spark.executor.memory","4g") \
        .getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [7]:
start_time = time.time()

# Read book file 
text_file = sc.textFile(book_file)

counts = text_file.flatMap(lambda line: line.split()) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

counts.toDF(["Word", "Count"]).sort("Count",ascending=False).show(5)

print("--- Parallel Execution time: %s seconds ---" % (time.time() - start_time))

+----+-----+
|Word|Count|
+----+-----+
| the|31704|
| and|20564|
|  to|16320|
|  of|14855|
|   a|10018|
+----+-----+
only showing top 5 rows

--- Parallel Execution time: 5.658057451248169 seconds ---


In [8]:
spark.stop()

### Five Data Partitions

In [9]:
# Starting Spark Cluster

spark = SparkSession.builder \
        .master("spark://lasidcluster:7077") \
        .appName("WordCountParallel_part") \
        .config("spark.sql.shuffle.partitions","10000") \
        .config("spark.driver.memory","4g") \
        .config("spark.executor.memory","4g") \
        .getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [10]:
start_time = time.time()

# Read book file 
part = 5 
text_file_part = sc.textFile(book_file, part)

counts = text_file_part.flatMap(lambda line: line.split()) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

counts.toDF(["Word", "Count"]).sort("Count",ascending=False).show(5)

print("--- Parallel Execution (partition) time: %s seconds ---" % (time.time() - start_time))

+----+-----+
|Word|Count|
+----+-----+
| the|31704|
| and|20564|
|  to|16320|
|  of|14855|
|   a|10018|
+----+-----+
only showing top 5 rows

--- Parallel Execution (partition) time: 5.446131467819214 seconds ---


In [11]:
spark.stop()